In [6]:
import os
from speechbrain.pretrained import SepformerSeparation as separator
import soundfile
import torchaudio
from pydub import AudioSegment
from IPython.display import Audio 
from IPython.core.display import display
import os
from scipy.io import wavfile
import noisereduce as nr
import os
import sys
import torch
import torch.nn.functional as F
import torchaudio
import speechbrain as sb
import speechbrain.nnet.schedulers as schedulers
import numpy as np
from tqdm import tqdm
import logging
from hyperpyyaml import load_hyperpyyaml
import csv
device = torch.device('cuda')
logger = logging.getLogger(__name__)

# from mir_eval.separation import bss_eval_sources
from speechbrain.dataio.dataio import read_audio
from fast_bss_eval import bss_eval_sources
import librosa

C:\Users\Arian\AppData\Local\Temp\ipykernel_26872\1103696278.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [7]:
import time
import concurrent.futures
def parse_audio(path):
   audio,sr = torchaudio.load(path)
   audio = torchaudio.functional.resample(audio, sr, 8000)
   # audio, sr = librosa.load(path,sr=8000)
   # audio = torch.from_numpy(audio).unsqueeze(0)

   # audio = audio
   # print(audio.shape)
   
   return audio
def append_audio(item_name, path, out_est, out_target, out_est_up, out_mix):
   # source1_path = f"{path}/{item_name}_source1hat_up.wav"
   # source2_path = f"{path}/{item_name}_source2hat_up.wav"
   # if not os.path.isfile(source1_path) or not os.path.isfile(source2_path):
   #    print(source1_path)
   #    return

   # Orig
   source1_path = f"{path}/{item_name}_source1.wav"
   source2_path = f"{path}/{item_name}_source2.wav"
   out_target[item_name] = [parse_audio(source1_path), parse_audio(source2_path)]

   source1_path = f"{path}/{item_name}_source1hat.wav"
   source2_path = f"{path}/{item_name}_source2hat.wav"
   out_est[item_name] = [parse_audio(source1_path), parse_audio(source2_path)]
   
   # source1_path = f"{path}/{item_name}_source1hat_up.wav"
   # source2_path = f"{path}/{item_name}_source2hat_up.wav"
   # out_est_up[item_name] = [parse_audio(source1_path), parse_audio(source2_path)]
   # print(out_est_up[item_name])

   mix_path = f"{path}/{item_name}_mix.wav"
   out_mix[item_name]=(parse_audio(mix_path))
   

def read_all_audio(path, upsampled=False):
   
   audio_ids = []
   out_target= {}
   out_est = {}
   out_est_upsampled = {}
   out_mix = {}
   all_files = os.listdir(path)
   item_names = [file_name.split('_')[0] for file_name in all_files] 
   item_names = list(set(item_names))
   
   # with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
   for item_name in tqdm(item_names, total=len(item_names)):
      append_audio(item_name, path, out_est, out_target, out_est_upsampled, out_mix)
      
   audio_ids =list(sorted(out_target.keys())) 
   out_target = list(dict(sorted(out_target.items())).values())
   out_est= list(dict(sorted(out_est.items())).values())
   out_est_upsampled= list(dict(sorted(out_est_upsampled.items())).values())
   out_mix= list(dict(sorted(out_mix.items())).values())
   print(audio_ids, out_target, out_est, out_est_upsampled)
   return audio_ids, out_target, out_est,out_est_upsampled, out_mix

In [8]:
import numpy as np
class Separation(sb.Brain):
   def compute_objectives(self, predictions, targets):
        """Computes the si-snr loss"""
        return self.hparams.loss(targets, predictions)
     
   def get_metrics(self,audio_ids, targets, preds, mixtures,output_path):

        # Create folders where to store audio
      # save_file = os.path.join(output_path, "test_results.csv")
      save_file = output_path

        # Variable init

      all_sdrs = []
      all_sdrs_i = []
      all_sisnrs = []
      all_sisnrs_i = []
      csv_columns = ["snt_id", "sdr", "sdr_i", "si-snr", "si-snr_i"]

        
      
      with open(save_file, "w") as results_csv:
         writer = csv.DictWriter(results_csv, fieldnames=csv_columns)
         writer.writeheader()
            
         for audio_id, target, pred,mixture_ in tqdm(zip(audio_ids,targets, preds,mixtures), total=len(targets)):
            target = torch.cat(
            [target[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
            
            pred = torch.cat(
            [pred[i].unsqueeze(-1) for i in range(self.hparams.num_spks)],
            dim=-1,
        ).to(self.device)
            
  
            sisnr = self.compute_objectives(pred, target)
            # COmpute SI-SNR Improvement
            mixture_signal = torch.stack(
               [mixture_] * self.hparams.num_spks, dim=-1
            )
            
            arrs = [pred[0], target[0], mixture_signal[0]]
            for item in arrs:
               temp_item = (item!=np.inf).all() and (item!=np.NINF).all()
               if not temp_item:
                  continue
            # if mixture_signal[0] == np.inf or mixture_signal[1] ==np.inf:
            
            mixture_signal = mixture_signal.to(target.device)
            sisnr_baseline = self.compute_objectives(
               mixture_signal, target
            )
            sisnr_i = sisnr.cpu().numpy() - sisnr_baseline.cpu().numpy()
  # Compute SDR
            sdr, _, _, _ = bss_eval_sources(
               target[0].t(),
               pred[0].t(),
            )

            sdr_baseline, _, _, _ = bss_eval_sources(
               target[0].t(),
               mixture_signal[0].t(),
            )
            
            sdr = sdr.cpu().numpy()
            sdr_baseline = sdr_baseline.cpu().numpy()
           
            sdr_i = sdr.mean() - sdr_baseline.mean()

            # Saving on a csv file
            row = {
               "snt_id": audio_id,
               "sdr": sdr.mean(),
               "sdr_i": sdr_i,
               "si-snr": -sisnr.item(),
               "si-snr_i": -sisnr_i.item(),
            }
            writer.writerow(row)

            # Metric Accumulation
            all_sdrs.append(sdr.mean())
            all_sdrs_i.append(sdr_i.mean())
            all_sisnrs.append(-sisnr.item())
            all_sisnrs_i.append(-sisnr_i.item())
      # logger.info("Mean SISNR is {}".format(np.array(all_sisnrs).mean()))
      # logger.info("Mean SISNRi is {}".format(np.array(all_sisnrs_i).mean()))
      # logger.info("Mean SDR is {}".format(np.array(all_sdrs).mean()))
      # logger.info("Mean SDRi is {}".format(np.array(all_sdrs_i).mean()))   


In [9]:
# Load hyperparameters file with command-line overrides
hparams_file, run_opts, overrides = sb.parse_arguments(["hyperparams.yaml"])
hparams_file = 'hyperparams.yaml'
#    run_opts = {"device": "cuda:0"}
with open(hparams_file) as fin:
   hparams = load_hyperpyyaml(fin, overrides)
   
# Load pretrained model if pretrained_separator is present in the yaml
if "pretrained_separator" in hparams:
   # run_on_main(hparams["pretrained_separator"].collect_files)
   hparams["pretrained_separator"].load_collected(
      device=run_opts["device"]
   )
# Brain class initialization
separator = Separation(
   modules=hparams["modules"],
   run_opts={"device": "cuda"},
   hparams=hparams,
)

configs = ["standard_model"]
# configs = ["w_noise_speedperturb","w_noise_wavedrop"]
# configs = ["no_noise_speedperturb"]
UPSAMPLE= [False]
# UPSAMPLE= [True]
curr_aud =[]
for config in configs:
   print(f"Reading {config}")
   # out = read_all_audio(f'results/{config}/audio_results')
   # audio_ids,out_target, out_est, out_est_upsampled, out_mix = out
   # audio_ids,out_target, out_est, out_est_upsampled, out_mix = read_all_audio(f'results/{config}/audio_results')
   audio_ids,out_target, out_est, out_est_upsampled, out_mix = read_all_audio(f'./separated')
   
   
   for is_upsample in UPSAMPLE:
      if is_upsample:
         out_target = out_est
         out_est = out_est_upsampled   
      # upsample = "upsampled" if upsample else "not_upsampled"
      output_path = f"test.csv"
      separator.get_metrics(audio_ids, out_target, out_est, out_mix, output_path=output_path)
      
      print(f"Done with {config} | upsamle: {is_upsample}")
# output_path = f"results/epoch_30/true_test_result.csv"
# separator.get_metrics(audio_ids, out_target, out_est, out_mix, output_path=output_path)
# separator.get_metrics(audio_ids, out_est, out_est_upsampled, out_mix, output_path=output_path)

Reading standard_model


100%|██████████| 2/2 [00:00<00:00,  4.85it/s]

Done with standard_model | upsamle: False


In [10]:
# model = separator.from_hparams(source="speechbrain/sepformer-libri2mix", savedir=f'./models/standard_model',run_opts={"device":"cuda"})
# est_sources = models[model_name].separate_file(path=f'{mix_path}/{file}') 
      
# item_name = file.split("_")[0]
# display(Audio(f"{mix_path}/{file}"))
# audio1_filename = f"{output_path}/{item_name}_source1hat.wav"
# audio2_filename = f"{output_path}/{item_name}_source2hat.wav"

# torchaudio.save(audio1_filename, est_sources[:, :, 0].detach().cpu(), 8000)
# torchaudio.save(audio2_filename, est_sources[:, :, 1].detach().cpu(), 8000)